In [1]:
import numpy as np
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import train_test_split

In [2]:
train_transform = transforms.Compose([
    transforms.Resize((256)), # resize all images 
    transforms.CenterCrop(224),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), 
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
inference_transform = transforms.Compose([
    transforms.Resize((256)), # resize all images 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
data_dir = 'D:\\Images\\Training_images\\data'

train_dataset = datasets.ImageFolder(data_dir, transform=train_transform)
val_test_dataset = datasets.ImageFolder(data_dir, transform=inference_transform)

In [5]:
# Extract indices and corresponding labels
indices = np.arange(len(train_dataset))
labels = np.array([label for _, label in train_dataset.imgs])

# Split into training (70%) and combined validation/test (30%)
train_indices, val_test_indices, train_labels, val_test_labels = train_test_split(
    indices, labels, test_size=0.3, stratify=labels, random_state=42)

# Split the combined validation/test set into validation (15%) and test (15%)
val_indices, test_indices, _, _ = train_test_split(
    val_test_indices, val_test_labels, test_size=0.5, stratify=val_test_labels, random_state=42)

# Create samplers for each set
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

# Create DataLoader for each set
train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_sampler, num_workers=2)
val_loader = DataLoader(val_test_dataset, batch_size=32, sampler=val_sampler, num_workers=2)
test_loader = DataLoader(val_test_dataset, batch_size=32, sampler=test_sampler, num_workers=2)

In [6]:
import timm

model = timm.create_model('vit_base_patch16_224', pretrained = True, num_classes = 22)

for param in model.parameters():
    param.requires_grad = False

for param in model.head.parameters():
    param.requires_grad = True

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

c:\Users\sirifris\.conda\envs\poli_reco\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import torch.optim as optim

model.head = torch.nn.Sequential(
    torch.nn.Dropout(0.5),
    torch.nn.Linear(model.head.in_features, 22)
)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [8]:
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [9]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to('cuda'), labels.to('cuda')

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [29]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\sirifris\.conda\envs\poli_reco\lib\site-packages\torch\utils\data\_utils\worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "c:\Users\sirifris\.conda\envs\poli_reco\lib\site-packages\torch\utils\data\_utils\fetch.py", line 55, in fetch
    return self.collate_fn(data)
  File "c:\Users\sirifris\.conda\envs\poli_reco\lib\site-packages\torch\utils\data\_utils\collate.py", line 398, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "c:\Users\sirifris\.conda\envs\poli_reco\lib\site-packages\torch\utils\data\_utils\collate.py", line 211, in collate
    return [
  File "c:\Users\sirifris\.conda\envs\poli_reco\lib\site-packages\torch\utils\data\_utils\collate.py", line 212, in <listcomp>
    collate(samples, collate_fn_map=collate_fn_map)
  File "c:\Users\sirifris\.co